# Notebook 04: Build Connectome Graph

This notebook converts the functional connectivity matrix into a NetworkX graph structure. A connectome graph represents brain regions as nodes and their functional connections as edges. The graph enables network analysis including centrality measures, community detection, and global network properties.

Key steps:
1. Load functional connectivity matrix from Notebook 03
2. Apply correlation threshold to define significant connections
3. Build NetworkX graph from thresholded connectivity matrix
4. Compute and visualize network statistics
5. Save graph in GraphML format for future use
6. Create degree distribution visualization

In [ ]:
# Import all required libraries for graph analysis
# numpy: numerical operations and array manipulation
# pandas: data structures and analysis
# json: load configuration
# Path: cross-platform file path handling
# networkx: graph/network analysis and algorithms
# scipy: correlation and scientific computing
# matplotlib: visualization
# numpy.ma: masked array operations for handling NaN values
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
import networkx as nx
from scipy.stats import threshold
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("All required libraries imported successfully")

In [ ]:
# Set random seed for reproducibility
# Ensures consistent results across multiple runs
np.random.seed(42)

print("Random seed set to 42 for reproducibility")

In [ ]:
# Load configuration and set up directory paths
notebook_dir = Path.cwd()
project_dir = notebook_dir.parent
data_dir = project_dir / 'data'
results_dir = project_dir / 'results'
figures_dir = project_dir / 'figures'

# Load config.json containing dataset parameters
config_path = data_dir / 'config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

n_regions = config['n_regions']

print(f"Configuration loaded from {config_path}")
print(f"Number of brain regions: {n_regions}")

In [ ]:
# Load the functional connectivity matrix from Notebook 03
# This matrix represents pairwise correlation strengths between all brain regions
# Shape: (n_regions, n_regions)
# Value at [i,j] = correlation coefficient between region i and region j

connectivity_path = data_dir / 'functional_matrix.npy'
print(f"Loading functional connectivity matrix from {connectivity_path}")

functional_matrix = np.load(connectivity_path)

print(f"\nFunctional Connectivity Matrix Properties:")
print(f"  Shape: {functional_matrix.shape}")
print(f"  Data type: {functional_matrix.dtype}")
print(f"  Min value: {np.nanmin(functional_matrix):.4f}")
print(f"  Max value: {np.nanmax(functional_matrix):.4f}")
print(f"  Mean value: {np.nanmean(functional_matrix):.4f}")
print(f"  Std value: {np.nanstd(functional_matrix):.4f}")

In [ ]:
# Apply threshold to define significant connections
# Thresholding is important because:
# 1. Very weak correlations (r < 0.3) may be due to noise
# 2. Creates sparsity - makes the graph more interpretable
# 3. Focuses on biologically meaningful connections

# Define threshold value
correlation_threshold = 0.3
print(f"Applying correlation threshold: r > {correlation_threshold}")
print(f"\nThreshold rationale:")
print(f"  - Removes weak correlations likely due to noise")
print(f"  - Focuses on moderate-to-strong functional connections")
print(f"  - Value of 0.3 is standard in resting-state fMRI analysis")
print(f"  - Creates sparse graph for efficient computation")

# Create a copy to preserve original
thresholded_matrix = functional_matrix.copy()

# Set all correlations below threshold to 0
# Keep diagonal as 0 (self-connections not meaningful)
thresholded_matrix[np.abs(thresholded_matrix) < correlation_threshold] = 0
np.fill_diagonal(thresholded_matrix, 0)  # Remove self-loops

print(f"\nThresholding Results:")
# Count connections above threshold
n_connections = np.sum(thresholded_matrix != 0)
max_possible_connections = n_regions * (n_regions - 1)  # Exclude diagonal
sparsity = n_connections / max_possible_connections

print(f"  Connections above threshold: {n_connections}")
print(f"  Maximum possible connections: {max_possible_connections}")
print(f"  Graph sparsity: {sparsity:.2%}")
print(f"  Connectivity range: [{np.min(thresholded_matrix[thresholded_matrix!=0]):.4f}, {np.max(thresholded_matrix):.4f}]")

In [ ]:
# Build NetworkX graph from thresholded connectivity matrix
# NetworkX is a Python library for analyzing complex networks
# A Graph object stores nodes (brain regions) and edges (connections)

print("Building NetworkX graph from connectivity matrix...")

# Create empty undirected graph
# Undirected: connections are bidirectional (if A connects to B, B connects to A)
G = nx.Graph()

# Add all regions as nodes
# Each node represents one brain region
G.add_nodes_from(range(n_regions))

# Add edges for all connections above threshold
# Iterate through upper triangle of matrix (avoid duplicates)
for i in range(n_regions):
    for j in range(i+1, n_regions):
        # Check if connection strength exceeds threshold
        connection_strength = thresholded_matrix[i, j]
        if connection_strength > 0:
            # Add edge with weight equal to correlation strength
            # Weight can be used for weighted network analysis
            G.add_edge(i, j, weight=connection_strength)

print(f"Graph construction complete")
print(f"\nGraph Properties:")
print(f"  Number of nodes: {G.number_of_nodes()}")
print(f"  Number of edges: {G.number_of_edges()}")
print(f"  Graph density: {nx.density(G):.4f}")
print(f"    (Density = edges / max_edges, ranges 0-1)")

In [ ]:
# Compute graph connectivity statistics
# These metrics describe the overall structure of the network

print("\n" + "="*70)
print("GRAPH STATISTICS")
print("="*70)

# Compute degree for each node
# Degree = number of connections for a node
degree_list = dict(G.degree())
degree_values = list(degree_list.values())

print(f"\nDegree Statistics:")
print(f"  Mean degree: {np.mean(degree_values):.2f}")
print(f"  Median degree: {np.median(degree_values):.2f}")
print(f"  Min degree: {np.min(degree_values)}")
print(f"  Max degree: {np.max(degree_values)}")
print(f"  Std dev: {np.std(degree_values):.2f}")

# Compute number of connected components
# A connected component is a group of nodes that are connected to each other
n_components = nx.number_connected_components(G)
print(f"\nConnectivity:")
print(f"  Number of connected components: {n_components}")
if n_components == 1:
    print(f"  Status: Network is fully connected")
    is_connected = True
else:
    print(f"  Status: Network has {n_components} disconnected subnetworks")
    is_connected = False
    # Show sizes of each component
    components = list(nx.connected_components(G))
    component_sizes = [len(c) for c in components]
    print(f"  Component sizes: {sorted(component_sizes, reverse=True)}")

# Compute clustering coefficient
# Measures how likely it is that neighbors of a node are also connected
# Range: 0-1 (0=no clustering, 1=complete clustering)
avg_clustering = nx.average_clustering(G)
print(f"\nNetwork Structure:")
print(f"  Average clustering coefficient: {avg_clustering:.4f}")
print(f"    (Measures how tightly grouped neighbors are)")

# Compute average shortest path length (if connected)
if is_connected:
    avg_path_length = nx.average_shortest_path_length(G)
    print(f"  Average shortest path length: {avg_path_length:.2f}")
    print(f"    (Average steps between any two regions)")
    
    # Compute network diameter
    diameter = nx.diameter(G)
    print(f"  Network diameter: {diameter}")
    print(f"    (Maximum shortest path in network)")
else:
    print(f"  Average shortest path length: Not computed (disconnected graph)")
    print(f"  Network diameter: Not computed (disconnected graph)")

In [ ]:
# Save graph in GraphML format for use in other notebooks
# GraphML is an XML-based format that preserves all graph information
# Advantages: human-readable, preserves node/edge attributes, widely supported

graph_output_path = results_dir / 'connectome_graph.graphml'
nx.write_graphml(G, graph_output_path)

print(f"\nGraph saved to: {graph_output_path}")
print(f"File size: {graph_output_path.stat().st_size / 1024:.2f} KB")

# Also save the thresholded connectivity matrix for reference
connectivity_output_path = results_dir / 'thresholded_connectivity_matrix.npy'
np.save(connectivity_output_path, thresholded_matrix)
print(f"\nThresholded connectivity matrix saved to: {connectivity_output_path}")

In [ ]:
# Create degree distribution visualization
# The degree distribution shows how many nodes have each degree value
# Important for understanding network organization

print("Creating degree distribution visualization...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Brain Connectome Network Analysis', fontsize=14, fontweight='bold')

# Plot 1: Degree distribution histogram
# Shows frequency distribution of node degrees
ax = axes[0, 0]
ax.hist(degree_values, bins=20, color='steelblue', alpha=0.7, edgecolor='black')
ax.axvline(np.mean(degree_values), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(degree_values):.2f}')
ax.set_xlabel('Node Degree')
ax.set_ylabel('Frequency')
ax.set_title('Degree Distribution')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 2: Cumulative degree distribution
# Shows what proportion of nodes have degree <= each value
ax = axes[0, 1]
sorted_degrees = sorted(degree_values)
cumulative = np.arange(1, len(sorted_degrees) + 1) / len(sorted_degrees)
ax.plot(sorted_degrees, cumulative, marker='o', markersize=4, linestyle='-', linewidth=2, color='darkgreen')
ax.set_xlabel('Node Degree')
ax.set_ylabel('Cumulative Proportion')
ax.set_title('Cumulative Degree Distribution')
ax.grid(True, alpha=0.3)

# Plot 3: Connection strength distribution
# Shows the distribution of edge weights (correlation strengths)
ax = axes[1, 0]
edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
ax.hist(edge_weights, bins=30, color='coral', alpha=0.7, edgecolor='black')
ax.axvline(np.mean(edge_weights), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(edge_weights):.3f}')
ax.set_xlabel('Edge Weight (Correlation)')
ax.set_ylabel('Frequency')
ax.set_title('Connection Strength Distribution')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 4: Network summary statistics
# Display key metrics as text
ax = axes[1, 1]
ax.axis('off')

summary_text = f"""NETWORK STATISTICS SUMMARY

Basic Properties:
  Nodes: {G.number_of_nodes()}
  Edges: {G.number_of_edges()}
  Density: {nx.density(G):.4f}

Degree:
  Mean: {np.mean(degree_values):.2f}
  Median: {np.median(degree_values):.2f}
  Range: {np.min(degree_values)}-{np.max(degree_values)}

Structure:
  Clustering coeff: {avg_clustering:.4f}
  Connected comp: {n_components}
  Threshold: r > {correlation_threshold}

Edge Weights:
  Mean: {np.mean(edge_weights):.4f}
  Range: {np.min(edge_weights):.4f}-{np.max(edge_weights):.4f}
"""

ax.text(0.1, 0.95, summary_text, transform=ax.transAxes, 
        fontsize=10, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plot_path = figures_dir / 'connectome_graph_analysis.png'
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"Visualization saved to {plot_path}")
plt.show()

In [ ]:
# Create final summary of graph analysis
print("\n" + "="*80)
print("CONNECTOME GRAPH CONSTRUCTION SUMMARY")
print("="*80)

print(f"\nInput Data:")
print(f"  Functional connectivity matrix shape: {functional_matrix.shape}")
print(f"  Correlation range: [{np.nanmin(functional_matrix):.4f}, {np.nanmax(functional_matrix):.4f}]")

print(f"\nThresholding Details:")
print(f"  Threshold method: Absolute correlation > 0.3")
print(f"  Rationale: Removes weak correlations (noise), focuses on strong connections")
print(f"  Self-loops: Removed (diagonal set to 0)")
print(f"  Resulting edges: {G.number_of_edges()} connections")

print(f"\nGraph Structure:")
print(f"  Type: Undirected, weighted network")
print(f"  Nodes: {G.number_of_nodes()} brain regions")
print(f"  Edges: {G.number_of_edges()} functional connections")
print(f"  Density: {nx.density(G):.4f} ({nx.density(G)*100:.2f}% of possible connections)")

print(f"\nNetwork Properties:")
print(f"  Average degree: {np.mean(degree_values):.2f}")
print(f"  Clustering coefficient: {avg_clustering:.4f}")
print(f"  Connected components: {n_components}")
if is_connected:
    print(f"  Average path length: {avg_path_length:.2f}")
    print(f"  Diameter: {diameter}")

print(f"\nWhat This Graph Represents:")
print(f"  - NODES: Brain regions (Destrieux atlas, n={n_regions})")
print(f"  - EDGES: Functional connections (Pearson correlation > 0.3)")
print(f"  - WEIGHTS: Strength of functional coupling (correlation values)")
print(f"\nBiological Interpretation:")
print(f"  - Connectome shows functional organization of brain")
print(f"  - Hubs (high degree) = integrative regions")
print(f"  - High clustering = local specialization")
print(f"  - Low path length = global integration")

print(f"\nOutput Files:")
print(f"  Graph (GraphML): {graph_output_path}")
print(f"  Thresholded matrix: {connectivity_output_path}")
print(f"  Visualization: {plot_path}")

print(f"\nNext Steps:")
print(f"  1. Compute network centrality measures (Notebook 05)")
print(f"  2. Identify hub regions")
print(f"  3. Detect community structure")
print(f"\n" + "="*80)